In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from scipy.signal import savgol_filter
from random import choice
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model
%matplotlib widget


In [2]:
trajectories = pd.read_csv('211004_ExNo_72_FM4-64_60x__df_total.csv')

In [4]:
trajectories

,timepoint,frame,time[s],x_position[nm],y_position[nm],x_displacement[nm],y_displacement[nm],Euclidean_p2p_distance[nm],Speed[um/s],Velocity[um/s],...,Cumulative_Euclidean_distance_2D,Cumulative_distance_along_spline_1D,Orthogonal_distance_to_spline,Transposed_euclidean_dist_p2p[nm],Displ_along_spline_p2p[nm],Ort_displacements_p2p,P2P_Velocities_Cumulative_Euclidean_distance,P2P_Velocities_Distance_along_spline,Dataset,Molecule#
0,0,1.0,0.000000,10787.821934,126789.966297,0.000000,0.000000,NaN,NaN,NaN,...,0.000000,245891.049020,22086.417969,NaN,345.450386,19.395609,1613.467943,1658.626241,211004_ExNo_72_FM4-64_60x_,0
1,1,2.0,0.050000,10806.844655,126597.301074,-19.022721,192.665223,193.602045,3.872041,3.872041,...,69.716255,246236.499405,22093.402344,-69.716255,0.000000,81.525561,2544.448957,2320.884056,211004_ExNo_72_FM4-64_60x_,0
2,2,3.0,0.100000,10780.758076,126311.530418,26.086579,285.770656,286.958843,5.739177,5.739177,...,172.303940,246236.499405,22122.759766,-102.587685,345.330694,-140.173414,2382.670485,2414.783562,211004_ExNo_72_FM4-64_60x_,0
3,3,4.0,0.150000,10988.611373,125987.858051,-207.853297,323.672367,384.664522,7.693290,7.693290,...,297.752228,246581.830100,22072.283203,-125.448288,690.312134,258.391172,2710.829586,2445.288547,211004_ExNo_72_FM4-64_60x_,0
4,4,5.0,0.200000,10842.389713,125403.353789,146.221660,584.504262,602.516395,12.050328,12.050328,...,499.706818,247272.142234,22165.330078,-201.954590,344.986346,43.786564,2888.454491,2760.302730,211004_ExNo_72_FM4-64_60x_,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58475,62,2413.0,120.599998,21296.806220,84942.328193,-16.043684,348.985373,349.353961,6.987079,6.987079,...,8514.083984,289618.186087,21365.695312,-124.472656,349.757065,-162.758234,2376.246954,2597.078513,211004_ExNo_72_FM4-64_60x_,999
58476,63,2414.0,120.650002,21521.434062,84633.050666,-224.627842,309.277527,382.243713,7.644874,7.644874,...,8638.948242,289967.943151,21307.085938,-124.864258,700.089578,-174.310988,1212.497854,1331.556703,211004_ExNo_72_FM4-64_60x_,999
58477,64,2416.0,120.750000,21810.788513,84044.489381,-289.354451,588.561285,655.843262,13.116865,13.116865,...,8867.402344,290668.032730,21244.316406,-228.454102,0.000000,-7.745227,2986.511728,2856.857780,211004_ExNo_72_FM4-64_60x_,999
58478,65,2417.0,120.800003,21840.733468,83927.546223,-29.944955,116.943158,120.716207,2.414324,2.414324,...,8910.825195,290668.032730,21241.527344,-43.422852,350.337522,38.254260,2429.786256,1682.245107,211004_ExNo_72_FM4-64_60x_,999


In [3]:
molecules = trajectories['Molecule#'].unique()

In [7]:
size_min = 100
long_traj = [molecule for molecule in molecules if len(trajectories.loc[trajectories['Molecule#']==molecule])>=size_min]

138

In [6]:
window = 25
fig, ax = plt.subplots()
for molecule in molecules[:100]:
    to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)&(trajectories['timepoint']<=window)]
    if len(to_plot)==window+1:
        to_plot.plot('x_position[nm]','y_position[nm]',ax=ax)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(2438.4245633239743, 35336.63558059693, 12707.64020500489, 142110.45827595217)

In [18]:
fig, ax = plt.subplots()

molecule = choice(molecules)
to_plot = trajectories.loc[trajectories['Molecule#']==molecule]
to_plot.plot('x_position[nm]','y_position[nm]',ax=ax)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(25944.378097900386, 26732.797957763672, 60843.178322656255, 68090.6211265625)

In [19]:


Y = to_plot['y_position[nm]']
X = to_plot['x_position[nm]']
X = np.array(X).reshape(-1,1)
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
model.coef_

array([-9.91913056])

In [24]:
fig, ax = plt.subplots()

x = 'x_position[nm]'
y = 'y_position[nm]'
to_plot = trajectories.loc[trajectories['Molecule#']==molecule]
to_plot.plot('x_position[nm]','y_position[nm]',ax=ax,kind="scatter",s=20)
ax.plot(X,model.predict(X),color='red')
# ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, ax = plt.subplots()

x = 'x_position[nm]'
y = 'y_position[nm]'
to_plot = trajectories.loc[trajectories['Molecule#']==molecule]
ax.scatter(X,(np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2),s = 20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
min_max = []
speeds = []
window = 25
for molecule in molecules:
    to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)&(trajectories['timepoint']<=window)]
    if len(to_plot)==window+1:
        print(len(trajectories.loc[trajectories['Molecule#']==molecule]))

        Y = to_plot['y_position[nm]']
        X = to_plot['x_position[nm]']
        X = np.array(X).reshape(-1,1)
        lm = linear_model.LinearRegression()
        model = lm.fit(X,Y)
        displacement = (np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2)
        tube_size = np.max(displacement)-np.min(displacement)
        speeds.append(to_plot['Velocity[um/s]'].mean())
        min_max.append(tube_size)

85
53
90
52
102
31
151
33
130
41
57
63
148
125
26
33
36
110
98
33
117
37
91
54
32
56
36
59
83
80
34
75
28
31
28
37
58
29
44
36
40
41
40
32
30
145
46
55
28
27
52
47
72
31
39
45
39
34
27
108
97
39
28
62
31
52
175
44
38
26
61
117
50
49
67
48
36
37
70
54
30
54
51
35
44
27
36
39
41
98
41
40
31
189
62
221
184
55
32
33
103
64
30
61
32
31
71
28
72
44
81
66
50
31
34
74
125
33
29
84
30
51
82
38
32
34
41
141
103
75
57
70
27
311
142
63
60
60
65
207
27
40
156
45
31
27
43
48
43
42
53
27
31
47
33
93
131
28
100
82
411
198
30
61
49
27
125
109
35
56
66
27
45
28
78
91
38
27
80
29
73
65
37
63
32
79
54
35
254
54
109
29
59
64
29
38
81
56
38
129
55
32
92
27
57
101
37
39
102
28
114
65
259
159
47
28
27
27
82
40
33
30
114
40
42
119
88
35
33
27
26
55
83
54
62
72
50
66
47
195
29
29
82
64
85
168
34
40
61
51
61
31
97
39
36
88
26
56
48
109
30
93
64
31
68
113
41
60
116
62
30
51
132
30
96
84
78
96
38
32
32
54
44
38
35
44
74
35
43
31
35
33
32
32
27
32
243
36
62
94
60
27
28
27
39
27
34
60
33
33
58
58
86
51
33
50
47
40
8

In [39]:
molecule = choice(long_traj)
to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)]
Y = to_plot['y_position[nm]']
X = to_plot['x_position[nm]']
X = np.array(X).reshape(-1,1)
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)
displacement = np.transpose((np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2))
msd = []
for tau in range(size_min//2):
    sd = []
    for index in range(len(displacement)-tau):
        sd.append((displacement[index]-displacement[index+tau])**2)
    msd.append(np.mean(sd))

In [44]:
fig, ax = plt.subplots()
for i in range(20):
    molecule = choice(long_traj)
    to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)]
    Y = to_plot['y_position[nm]']
    X = to_plot['x_position[nm]']
    X = np.array(X).reshape(-1,1)
    lm = linear_model.LinearRegression()
    model = lm.fit(X,Y)
    displacement = np.transpose((np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2))
    msd = []
    for tau in range(size_min//2):
        sd = []
        for index in range(len(displacement)-tau):
            sd.append((displacement[index]-displacement[index+tau])**2)
        msd.append(np.mean(sd))
    ax.plot(np.sqrt(msd),alpha = 0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
displacement

array([[-685.4146767 ],
       [-232.09132692],
       [ -13.81137146],
       [  27.84308568],
       [ 310.14097207],
       [ 136.50873034],
       [ 155.30708032],
       [ 114.82091143],
       [ 109.58156616],
       [ 131.19208196],
       [ 126.5706963 ],
       [ 142.74491989],
       [ 103.63396139],
       [ -50.20908144],
       [-105.9055084 ],
       [-105.64168237],
       [-101.30060034],
       [-159.44991649],
       [-128.0479859 ],
       [-154.10850082],
       [-152.34734197],
       [-215.60033354],
       [-197.48916617],
       [-255.26974759],
       [-216.64698398],
       [-281.34733363],
       [-319.3112503 ],
       [-271.41759502],
       [-234.71194978],
       [-135.53035821],
       [ -14.18718885],
       [ -98.44696054],
       [-171.7443365 ],
       [-186.99189954],
       [-150.15352601],
       [ -75.94048145],
       [-164.44543688],
       [-106.12476893],
       [ -96.97778786],
       [  62.57081079],
       [ -22.81509396],
       [ -81.526

In [45]:
min_max = {}
speeds = {}
window = 25
min_max[window] = []
for molecule in long_traj:
    to_plot = trajectories.loc[(trajectories['Molecule#']==molecule)]
    Y = to_plot['y_position[nm]']
    X = to_plot['x_position[nm]']
    X = np.array(X).reshape(-1,1)
    lm = linear_model.LinearRegression()
    model = lm.fit(X,Y)
    displacement = (np.transpose(X)*model.coef_-np.array(Y)+model.intercept_)/np.sqrt(1+model.coef_**2)


In [48]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
ax.scatter(np.abs(speeds),min_max,alpha = 0.2,s=30)
ax.set_ylim((0,2000))
ax.set_xlabel('mean velocity')
ax.set_ylabel('tube size')
ax2.hist(np.abs(speeds),20,alpha = 0.2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  5.,   7.,   9.,  12.,   6.,  33., 114., 260., 155.,  35.,  20.,
         16.,  12.,  18.,  19.,  25.,  19.,  19.,   6.,   4.]),
 array([ 0.05953972,  0.97439441,  1.88924909,  2.80410377,  3.71895845,
         4.63381314,  5.54866782,  6.4635225 ,  7.37837719,  8.29323187,
         9.20808655, 10.12294123, 11.03779592, 11.9526506 , 12.86750528,
        13.78235997, 14.69721465, 15.61206933, 16.52692401, 17.4417787 ,
        18.35663338]),
 <BarContainer object of 20 artists>)

In [49]:
len(speeds)

794

In [37]:
fig, ax = plt.subplots()
tube_sizes = [tube_size for tube_size in min_max if tube_size<=2000]
ax.hist(tube_sizes,20)
ax.set_xlabel('Transverse displacement(nm)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Transverse displacement(nm)')

(array([  6.,   4.,   8.,  15.,  69., 188., 156., 100.,  51.,  34.,  22.,
         25.,  14.,  16.,  19.,  26.,  20.,  13.,   3.,   5.]),
 array([ 2.88930052,  3.66266716,  4.43603381,  5.20940045,  5.98276709,
         6.75613373,  7.52950038,  8.30286702,  9.07623366,  9.84960031,
        10.62296695, 11.39633359, 12.16970024, 12.94306688, 13.71643352,
        14.48980016, 15.26316681, 16.03653345, 16.80990009, 17.58326674,
        18.35663338]),
 <BarContainer object of 20 artists>)

In [75]:
len(min_max)

794